# AQC per il problema dello zaino

In [215]:
import pandas as pd
import dimod
from sympy import Sum, IndexedBase, Idx, expand, Poly
from sympy.abc import m, c, lamda

In [216]:
data = pd.read_csv('data/very_small.csv', names=['p', 'w'])
data.index = range(1, len(data) + 1)

num_items = len(data)
capacity = int(0.8 * sum(data['w']))

data

,p,w
1,3,1
2,4,3
3,2,2


In [217]:
j = Idx('j')
n = Idx('n')
X = IndexedBase('x', shape=num_items)
P = IndexedBase('p', shape=num_items)
W = IndexedBase('w', shape=num_items)
Y = IndexedBase('y', shape=capacity)

syms = [X[i] for i in range(1, num_items + 1)] + [Y[i] for i in range(1, capacity + 1)]

expr = - Sum(P[j] * X[j], (j, 1, m)) + \
       lamda * ((1 - Sum(Y[n], (n, 1, c))) ** 2 + (Sum(n * Y[n], (n, 1, c)) - Sum(W[j] * X[j], (j, 1, m))) ** 2)

expr

lamda*((1 - Sum(y[n], (n, 1, c)))**2 + (Sum(n*y[n], (n, 1, c)) - Sum(w[j]*x[j], (j, 1, m)))**2) - Sum(p[j]*x[j], (j, 1, m))

In [218]:
expr = expr.subs([(m, num_items), (c, capacity)]).doit() \
           .subs([(P[i], data['p'][i]) for i in range(1, num_items + 1)]) \
           .subs([(W[i], data['w'][i]) for i in range(1, num_items + 1)]) \
           .subs(lamda, 1 + data['p'].max())

expr

5*(-y[1] - y[2] - y[3] - y[4] + 1)**2 + 5*(-x[1] - 3*x[2] - 2*x[3] + y[1] + 2*y[2] + 3*y[3] + 4*y[4])**2 - 3*x[1] - 4*x[2] - 2*x[3]

In [219]:
expr = expand(expr).replace(lambda x: x.is_Pow, lambda x: x.base)
poly = Poly(expr)

expr

30*x[1]*x[2] + 20*x[1]*x[3] - 10*x[1]*y[1] - 20*x[1]*y[2] - 30*x[1]*y[3] - 40*x[1]*y[4] + 2*x[1] + 60*x[2]*x[3] - 30*x[2]*y[1] - 60*x[2]*y[2] - 90*x[2]*y[3] - 120*x[2]*y[4] + 41*x[2] - 20*x[3]*y[1] - 40*x[3]*y[2] - 60*x[3]*y[3] - 80*x[3]*y[4] + 18*x[3] + 30*y[1]*y[2] + 40*y[1]*y[3] + 50*y[1]*y[4] + 70*y[2]*y[3] + 90*y[2]*y[4] + 15*y[2] + 130*y[3]*y[4] + 40*y[3] + 75*y[4] + 5

In [220]:
Q = pd.DataFrame(0, index=syms, columns=syms)

for sym1 in syms:
    for sym2 in syms[syms.index(sym1):]:
        Q.loc[sym1, sym2] = int(poly.coeff_monomial(sym1 if sym1 == sym2 else sym1*sym2))

Q

,x[1],x[2],x[3],y[1],y[2],y[3],y[4]
x[1],2,30,20,-10,-20,-30,-40
x[2],0,41,60,-30,-60,-90,-120
x[3],0,0,18,-20,-40,-60,-80
y[1],0,0,0,0,30,40,50
y[2],0,0,0,0,15,70,90
y[3],0,0,0,0,0,40,130
y[4],0,0,0,0,0,0,75


In [221]:
dimod.ExactSolver().sample_qubo(Q).to_pandas_dataframe().sort_values(by='energy')

,0,1,2,3,4,5,6,energy,num_occurrences
125,1,1,0,0,0,0,1,-12.0,1
57,1,0,1,0,0,1,0,-10.0,1
101,1,1,1,0,1,0,1,-9.0,1
60,0,1,0,0,0,1,0,-9.0,1
14,1,0,0,1,0,0,0,-8.0,1
...,...,...,...,...,...,...,...,...,...
79,0,0,0,1,0,1,1,335.0,1
87,0,0,1,1,1,1,1,358.0,1
95,0,0,0,0,1,1,1,420.0,1
81,1,0,0,1,1,1,1,442.0,1


In [224]:
dimod.SimulatedAnnealingSampler().sample_qubo(Q).to_pandas_dataframe().sort_values(by='energy')

,0,1,2,3,4,5,6,energy,num_occurrences
2,1,0,0,1,0,0,0,-8.0,1
3,0,0,1,0,1,0,0,-7.0,1
4,0,0,1,0,1,0,0,-7.0,1
0,0,1,1,0,1,1,0,-6.0,1
1,0,1,1,0,1,1,0,-6.0,1
6,0,1,1,0,1,1,0,-6.0,1
8,0,1,1,0,1,1,0,-6.0,1
5,1,1,1,0,0,1,1,-4.0,1
7,1,1,1,0,0,1,1,-4.0,1
9,1,1,1,0,0,1,1,-4.0,1


In [223]:
# bqm = BinaryQuadraticModel.from_qubo(QUBO_Q)
# qsampler = EmbeddingComposite(DWaveSampler())
# qsampler.sample(bqm, num_reads=20).to_pandas_dataframe().sort_values(by='energy')